In [1]:
!pip install openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import os
import json
import openai
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

openai.api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(api_key=openai.api_key, model="gpt-4")

fields = [
    {"name": "ID", "start": 0, "length": 5, "type": "numeric"},
    {"name": "Name", "start": 5, "length": 20, "type": "alphanumeric"},
    {"name": "Age", "start": 25, "length": 3, "type": "numeric"},
]

def generate_positional_data_as_json(prompt_text, num_records=10):
    prompt = PromptTemplate.from_template(prompt_text)
    
    generated_text = model.predict(prompt.format())
    
    positional_data_json = []
    
    for record_line in generated_text.split("\n")[:num_records]:
        record = {}
        for field in fields:
            start, length = field["start"], field["length"]
            field_value = record_line[start:start + length].strip()
            
            record[field["name"]] = {
                "value": field_value,
                "start": start,
                "length": length,
                "type": field["type"]
            }
        
        positional_data_json.append(record)
    
    with open("output_positional.json", "w") as file:
        json.dump(positional_data_json, file, indent=4)

# Exemplo de uso
prompt_text = """ 
Gere 10 registros no formato posicional, com os campos definidos abaixo. Cada registro deve ser uma linha separada por quebras de linha, sem delimitadores de campo. A cada linha, cada campo deve estar na posição exata definida e seguir as regras especificadas para comprimento, preenchimento e tipo de dados.

Campos e regras de formatação:

ID: 5 caracteres, numérico, começando na posição 0. Deve ser um número inteiro positivo e sempre ter 5 dígitos, preenchendo com zeros à esquerda se necessário (ex: 00001, 00123).
Nome: 20 caracteres, alfanumérico, começando na posição 5. Deve ser um texto contendo apenas letras e espaços, com um comprimento fixo de 20 caracteres. Caso o nome tenha menos de 20 caracteres, complete com espaços à direita (ex: Ana Silva ).
Idade: 3 caracteres, numérico, começando na posição 25. Deve ser um número inteiro positivo com 3 dígitos, preenchendo com zeros à esquerda se necessário (ex: 005, 035).
Formato de exemplo:

00001Ana Silva           025
00002João Oliveira       030
Certifique-se de que cada campo está corretamente posicionado e segue o tipo de dado especificado, com o comprimento fixo e o preenchimento adequado. Retorne o resultado como uma sequência de registros, com um total de 10 registros, cada um em uma linha.

"""
generate_positional_data_as_json(prompt_text)


In [12]:
def generate_positional_txt_from_json(json_file, output_txt_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    lines = []
    
    for record in data:
        line = ""
        for field in record.values():
            value = field["value"]
            length = field["length"]
            field_type = field["type"]

            if field_type == "numeric":
                formatted_value = str(value).zfill(length)
            else:
                formatted_value = str(value).ljust(length)

            line += formatted_value
        
        lines.append(line)
    
    with open(output_txt_file, 'w') as txt_file:
        txt_file.write("\n".join(lines))

generate_positional_txt_from_json("output_positional.json", "output_positional.txt")